In [1]:
import pandas as pd
import numpy as np
import sys, os
import seaborn as sns

from matplotlib import pyplot as plt
sys.path.insert(1, os.path.join(sys.path[0], '../../'))  # for importing local packages from src
sys.path.insert(1, os.path.join(sys.path[0], '../'))  # for importing model config

%reload_ext autoreload
%autoreload 2 

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath} \usepackage{gensymb} \usepackage{siunitx}'
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 12

## Datasets

In [2]:
from src.model.rc_dataset import ExperimentDataset

DATASET_PATH = '../datasets/hydroshoot_large_trimmed.csv'

dataset = ExperimentDataset(csv_path=DATASET_PATH)

## Model configuration

In [3]:
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GroupKFold
from src.learning.scorers import nmse_scorer
from pipeline_hydroshoot import GroupGenerator, TimeGenerator
from pipeline_base import TrainTestSplitter, DirectTransform, WarmupTransform

# constants
DATASET_ID = 'HydroShoot_large'
STATE_SIZE = 32


# Reservoir generation
np.random.seed(42)
state_ids = np.random.choice(dataset.state_size(), size=STATE_SIZE, replace=False)


# readout model
readout_model = Pipeline([
  ('ridge_regression', Ridge(alpha=1, fit_intercept=True))
])
model_param_grid = [{
  'ridge_regression__alpha': 10 ** np.linspace(np.log10(1e-4), np.log10(1e2), 50)
}]


# Regression task pipeline
datasets = (dataset,)
run_ids = dataset.get_run_ids()
shared_pipeline_params = {
  # Data generation
  'datasets': datasets,
  'groups': GroupGenerator(day_length=24, run_ids=run_ids, days_between_runs=1),
  'time': TimeGenerator(day_length=24, run_ids=run_ids),
  
  # Model training and validation
  'readout_model': readout_model,
  'model_param_grid': model_param_grid,
  'model_scorer': nmse_scorer,
  'folds': GroupKFold(n_splits=5),
  'train_test_split': TrainTestSplitter(block_size=4, test_ratio=0.5),
}

## Regression Pipelines

In [4]:
from pipeline_base import (
  RCPipeline, 
  Rescale, 
  DaylightMask
)

from pipeline_hydroshoot import (
  TargetGenerator, 
  SingleReservoirGenerator,
  MultiReservoirGenerator,
  TargetReservoirGenerator,
  GroupRescale
)


from model_config_hydroshoot import (
  targets, 
  measurable_reservoirs,
  baseline_reservoirs,
  heterogeneous_reservoirs, 
)

In [5]:
targets

['input_Tac',
 'input_u',
 'input_hs',
 'input_Rg',
 'output_Rg',
 'output_An',
 'output_E',
 'output_Tleaf']

In [17]:
measurable_reservoirs = ['state_Tlc', 'state_E', 'state_gs']
final_targets = ['input_Rg', 'output_Rg', 'output_An']
narma_targets = ['input_Rg']

target_state_pairs = [(target, state_var) for target in final_targets for state_var in measurable_reservoirs]
narma_pairs = [(target, state_var) for target in narma_targets for state_var in measurable_reservoirs]
target_env_pairs = []
target_het_pairs = []

print(targets)
print(measurable_reservoirs)
print(baseline_reservoirs)
print(heterogeneous_reservoirs)

['input_Tac', 'input_u', 'input_hs', 'input_Rg', 'output_Rg', 'output_An', 'output_E', 'output_Tleaf']
['state_Tlc', 'state_E', 'state_gs']
[('env_all', ['input_Tac', 'input_u', 'input_hs', 'input_Rg']), ('env_temp', ['input_Tac']), ('env_humidity', ['input_hs']), ('env_PAR', ['input_Rg']), ('env_wind', ['input_u'])]
[('state_all', ('state_An', 'state_Tlc', 'state_gs', 'state_E', 'state_Flux', 'state_psi_head'))]


In [13]:
from pipeline_base import NarmaTargetTransform

input_phys_transforms = [
  WarmupTransform(warmup_days=4, day_length=24),
  DirectTransform()
]


def generate_single(target_state_pairs, state_ids):
  for target, state_var in target_state_pairs:
    pipeline = RCPipeline(
      metadata={'target_id': target, 'reservoir_id': state_var, 'dataset_id': DATASET_ID},
      # Data generation
      target=TargetGenerator(target=target, run_ids=run_ids),
      reservoir=SingleReservoirGenerator(state_var=state_var, run_ids=run_ids, state_ids=state_ids),
      # Data transformation
      transforms=input_phys_transforms,
      # Data preprocessing
      preprocessing=[
        DaylightMask(day_length=24, start=5, end=21),
        Rescale(per_feature=False)
      ],
      **shared_pipeline_params,
    )
    yield pipeline


def generate_multi(target_het_pairs, state_ids):
  for name, target, state_vars in target_het_pairs:
    pipeline = RCPipeline(
      metadata={'target_id': target, 'reservoir_id': name, 'dataset_id': DATASET_ID},
      # Data generation
      target=TargetGenerator(target=target, run_ids=run_ids),
      reservoir=MultiReservoirGenerator(state_vars=state_vars, run_ids=run_ids, state_ids=state_ids),
      # Data transformation
      transforms=input_phys_transforms,
      # Data preprocessing
      preprocessing=[
        DaylightMask(day_length=24, start=5, end=21),
        GroupRescale(datasets=datasets, state_vars=state_vars, state_ids=state_ids)
      ],
      **shared_pipeline_params,
    )
    yield pipeline


def generate_env(target_env_pairs):
  for name, target, env_targets in target_env_pairs:
    pipeline = RCPipeline(
      metadata={'target_id': target, 'reservoir_id': name, 'dataset_id': DATASET_ID},
      # Data generation
      target=TargetGenerator(target=target, run_ids=run_ids),
      reservoir=TargetReservoirGenerator(targets=env_targets, run_ids=run_ids),
      # Data transformation
      transforms=input_phys_transforms,
      # Data preprocessing
      preprocessing=[
        DaylightMask(day_length=24, start=5, end=21),
        Rescale(per_feature=True)
      ],
      **shared_pipeline_params,
    )
    yield pipeline

def narma_benchmark(target_state_pairs, state_ids, *, n):
  for target, state_var in target_state_pairs:
    pipeline = RCPipeline(
      metadata={
        'target_id': f'{target}_NARMA_{n}', 
        'reservoir_id': state_var, 
        'dataset_id': DATASET_ID,
        'benchmark': 'NARMA',
        'narma_n': n,
      },
      # Data generation
      target=TargetGenerator(target=target, run_ids=run_ids),
      reservoir=SingleReservoirGenerator(state_var=state_var, run_ids=run_ids, state_ids=state_ids),
      # Data transformation
      transforms=[
        # NOTE: remove warmup before transform, otherwise the 
        # warmup steps are used as input to the NARMA system.
        WarmupTransform(warmup_days=4, day_length=24),  
        NarmaTargetTransform(n=n, scale=1),
      ],
      # Data preprocessing
      preprocessing=[
        DaylightMask(day_length=24, start=5, end=21),
        Rescale(per_feature=False)
      ],
      **shared_pipeline_params,
    )
    yield pipeline

In [29]:
from itertools import chain


def generate_pipelines(state_ids):
  pipelines = list(chain(
    generate_single(target_state_pairs, state_ids),
    generate_multi(target_het_pairs, state_ids),
    generate_env(target_env_pairs),
    narma_benchmark(narma_pairs, state_ids, n=8)
  ))
  return pipelines


pipelines = generate_pipelines(state_ids)
print(f'Total pipelines: {len(pipelines)}')

Total pipelines: 12


## Model fitting

In [30]:
from tqdm import tqdm
from itertools import chain
from pipeline_base import execute_pipeline



total_loops = len(pipelines)
results = []
model_data = []

with tqdm(total=total_loops) as pbar:
  for pipeline in pipelines:

    try:
      result, md = execute_pipeline(pipeline, return_model_data=True)
      results.append(result)
      model_data.append(md)
      pbar.update(1)
    except Exception as e:
      print('An exception occured executing the pipeline with the following metadata:')
      print(f'{pipeline.metadata}')
      raise e



100%|██████████| 12/12 [00:03<00:00,  3.29it/s]


## Prediction

In [31]:
hydroshoot_pred = {}


for result, md in zip(results, model_data):
  model = md['final_model']
  X_test, y_test = md['test_data']
  target_id = result['target_id']
  reservoir_id = result['reservoir_id']
  y_pred = model.predict(X_test)
  hydroshoot_pred[f'{target_id}__true'] = y_test
  hydroshoot_pred[f'{target_id}__{reservoir_id}'] = y_pred


list(hydroshoot_pred.keys())

['input_Rg__true',
 'input_Rg__state_Tlc',
 'input_Rg__state_E',
 'input_Rg__state_gs',
 'output_Rg__true',
 'output_Rg__state_Tlc',
 'output_Rg__state_E',
 'output_Rg__state_gs',
 'output_An__true',
 'output_An__state_Tlc',
 'output_An__state_E',
 'output_An__state_gs',
 'input_Rg_NARMA_8__true',
 'input_Rg_NARMA_8__state_Tlc',
 'input_Rg_NARMA_8__state_E',
 'input_Rg_NARMA_8__state_gs']

In [32]:
hs_pred_fixed = {}

for name, pred in hydroshoot_pred.items():
  l = len(pred)
  new_pred = np.ones(1968) * np.NaN
  new_pred[:l] = pred
  hs_pred_fixed[name] = new_pred
    

In [33]:
pred_df = pd.DataFrame(hs_pred_fixed)
pred_df.to_csv('results_hydroshoot_input_phys_predictions.csv')

pred_df.shape

(1968, 16)